# 华为手表使用前后入睡潜伏期数据处理

In [1]:
!pip install python-docx

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
import os
from docx import Document
import json
import re
from datetime import datetime
import pandas as pd

In [3]:
#判断小时位上是否小于12h，如果小于12小时加一天
def time_datetime(time):
    l = []
    for i in time:

        t = int(i.split(":")[0])
        if t <12:
            date = "2024-01-02"
        else:
            date = "2024-01-01"
        l.append(f'{date}-{i}')

    return l

In [4]:
# 差值转换成分钟
def total_time(x1,x2):
    minutess = []
    x1 = pd.to_datetime(x1)
    x2 = pd.to_datetime(x2)

    sleep_time = (x2-x1)
    for i in range(len(sleep_time)):
        m = int((sleep_time[i].total_seconds())//60)
        minutess.append(int(m))
    return minutess

In [5]:
#时间辍转化成标准时间
def transfer(time):
    time = datetime.fromtimestamp(time/1000000000)
    return time

In [6]:
before_huawei_bed_time = []
name_list = []
after_huawei_bed_time = []
before_psg_bed_time = []
after_psg_bed_time = []
s_before = []
s_after = []

In [7]:
path = "../data/第2晚已监测完"
dir = os.listdir(path)

In [8]:
#计算华为手表、PSG使用前后入睡潜伏期
for i in dir:
    dirpath = os.path.join(path,i)
    name = re.findall(r'[（](.*?)[）]',str(i))
    #计算华为手表使用前入睡潜伏期
    hauwei_before =  os.path.join(dirpath,"Before_Use/Huawei_Watch_Data/Sleep_Report.json")
    with open(hauwei_before,'r')as f:
        a = json.load(f)
        t = a["healthRecords"][0]['startTime']
        t = str(transfer(t))
        t =t[-8:-3]

        before_huawei_bed_time.append(t)
        name_list.append(name[0])
    # 计算华为手表使用后入睡潜伏期
    hauwei_after =  os.path.join(dirpath,"After_Use/Huawei_Watch_Data/Sleep_Report.json")
    with open(hauwei_after,'r')as f:
        a = json.load(f)
        t = a["healthRecords"][0]['startTime']
        t = str(transfer(t))
        t =t[-8:-3]

        after_huawei_bed_time.append(t)

# 读取PSG使用前入睡潜伏期
    psg_before =  os.path.join(dirpath,"Before_Use/Psg_Data/Hsy_Sleep_Report/Hsy_Sleep_Report1.docx")

    document = Document(psg_before)

    psg_before_text = (document.tables[1].rows[0].cells[0].text)[-5:]
    light_time = document.tables[1].rows[1].cells[2].text

    sleep_time = light_time[6:-1]
    s_before.append(sleep_time)

    before_psg_bed_time.append(psg_before_text)

# 读取PSG使用后入睡潜伏期
    psg_after =  os.path.join(dirpath,"After_Use/Psg_Data/Hsy_Sleep_Report/Hsy_Sleep_Report1.docx")

    document = Document(psg_after)

    psg_after_text = (document.tables[1].rows[0].cells[0].text)[-5:]

    light_time = document.tables[1].rows[1].cells[2].text
    sleep_time = light_time[6:-1]
    s_after.append(sleep_time)

    after_psg_bed_time.append(psg_after_text)


In [9]:
#转换成dataframe格式
df = pd.DataFrame()
df["姓名"] = name_list
df["PSG使用前关灯时间"] = before_psg_bed_time
df["华为手表使用前第一帧"]  = before_huawei_bed_time
df['PSG使用后关灯时间'] = after_psg_bed_time
df['华为手表使用后第一帧'] =after_huawei_bed_time
df['psg使用前入睡潜伏期'] = s_before
df['psg使用后入睡潜伏期'] = s_after

In [10]:

#计算使用前华为手表入睡潜伏期（华为手表使用前第一帧入睡时间-PSG使用前关灯时间）
x1 = time_datetime(df['PSG使用前关灯时间'])
x2 = time_datetime(df['华为手表使用前第一帧'])
y1 = total_time(x1,x2)
df["使用前相差"] =y1

#计算使用后华为手表入睡潜伏期（华为手表使用后第一帧入睡时间-PSG使用后关灯时间）
x1 = time_datetime(df['PSG使用后关灯时间'])
x2 = time_datetime(df['华为手表使用后第一帧'])
y1 = total_time(x1,x2)
df["使用后相差"] =y1

df.to_excel("../result/华为手表完整入睡潜伏期.xlsx")